Environment used is nlp

## Important note:

### To use language tool for spell check
1. Download the language-tool English file from https://www.languagetool.org/download/
2. Place the zipped file in C:\Users\<"username">\\.cache\language_tool_python
3. Make the following changes in language-tool package codes. This could be present in C:\Users\sbanerje111521\bin\anaconda3-2022.10\envs\nlp\Lib\site-packages\language_tool_python:
    - In **download_lt.py**:
        + Change `LATEST_VERSION = '5.7'` to `LATEST_VERSION = '5.6'`, or the version of the language-tool English file that you have.
        + Comment `download_zip(language_tool_download_url, download_folder)` at around line 156. This is done to bypass the above steps that is already completed.
    - In **which.py**:
        + Define `JAVA_PATH = "C:\\Users\\sbanerje111521\\bin\\jdk-18.0.2\\bin"` in beginning (around line 12). This is to manually set JAVA_HOME path.
        + Include `paths.append(JAVA_PATH)` statement in `get_path_list()` function definition (at around line 60). This is to include or append `JAVA_PATH` in path variable.

### To use `spacy` with English tokenizer `en_core_web_sm`
1. Install `spacy` package using `pip install spacy` or `conda install spacy` in command prompt.
2. Check the version of spacy version installed. To check version: `pip list` or `conda list` in command prompt.
3. Download the same version `en_core_web_sm` model zip file from https://github.com/explosion/spacy-models/releases. Unzip it.
4. Open your Python environment path. You can check by importing any package and typing `<package_name>.__path__` in Python
5. Save the folder containing __init__.py file in the site-packages folder.

## Import packages

In [ ]:
import matplotlib.pyplot as plt

# all functions are stored
from utilities import *

# to detect tense
# import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

# for grammatical check
from language_tool_python import LanguageTool

# to display progress bar
from tqdm import tqdm
tqdm.pandas()

## Define Control variables

In [ ]:
#INPUT_FULL_PATH = r"C:\Users\sbanerje111521\OneDrive - GROUP DIGITAL WORKPLACE\Documents\Soumya_docs\python\poc_final\inputs\BusinessDataDictionnary.xlsx"
INPUT_FULL_PATH = r"C:\Users\sbanerje111521\OneDrive - GROUP DIGITAL WORKPLACE\Documents\Soumya_docs\python\poc_final\inputs\Weekly_BMT_RISQ_6262023228AM.xlsx"
OUTPUT_PATH = r"C:\Users\sbanerje111521\OneDrive - GROUP DIGITAL WORKPLACE\Documents\Soumya_docs\python\poc_final\outputs"

SPECIAL_CHARS = ['$','%','&','*','@','#','!','`','~','^','€','¿']

REMOVE_NOTE = True

## Read & clean data

### Read data

In [ ]:
df = pd.read_excel(INPUT_FULL_PATH, sheet_name="AMER CDEs")
df.head(3)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

16 rows missing Golden Source

### Remove duplicates

In [ ]:
# Remove duplicates
df = df.drop_duplicates(subset=['CDE No'])
df = df.reset_index(drop=True)
df.head(3)

In [ ]:
df.shape

## Check Status

In [ ]:
df['CDE Status'].value_counts()

In [ ]:
# df = df[df['CDE Status']=='Active']
# df.shape

### Lower text

In [ ]:
# lower the text
df['lower_def'] = df['CDE Definition'].progress_apply(lambda x: x.lower())
df.head(3)

### Remove new line character

In [ ]:
df['new_line_def'] = df['lower_def'].progress_apply(lambda x: remove_new_line(x, REMOVE_NOTE))
df.head(3)

### Remove punctuation

In [ ]:
#storing the puntuation free text
df['clean_def']= df['new_line_def'].progress_apply(lambda x:remove_punctuation(x))
df.head(3)

### Stopwords removal

In [ ]:
#applying the function
df['no_stopwords']= df['clean_def'].progress_apply(lambda x: remove_stopwords(x))
df.head(3)

### Lemmatization

In [ ]:
# Lemmatization
df['lemmatized']= df['no_stopwords'].progress_apply(lambda x: lemmatizer(x))
df.head(3)

In [ ]:
import pickle
with open('lemmatized_archer_result.pkl', 'wb') as wfile:
    pickle.dump(df, wfile)

In [ ]:
import pickle
file = open("lemmatized_archer_result.pkl",'rb')
df = pickle.load(file)
df.head(3)

## Generate Metrics

### Count words

In [ ]:
df['no_of_words'] = df['new_line_def'].progress_apply(lambda x: count_words(x))
df.head(3)

### Count no. of characters

In [ ]:
df['no_of_chars'] = df['new_line_def'].progress_apply(lambda x: count_chars(x))
df.head(3)

### No. of special characters

In [ ]:
df[['list_spl_chars', 'count_spl_chars']] = df['new_line_def'].progress_apply(lambda x: count_spl_chars(x,SPECIAL_CHARS))
df.head(3)

### No. of Acronyms

In [ ]:
df[['list_acronym', 'count_acronym']] = df['CDE Definition'].progress_apply(lambda x: count_acronym(x))
df.head(3)

### To detect Present Tense

In [ ]:
df['is_past_tense'] = df['clean_def'].progress_apply(lambda x: detect_past_sentece(x))
df.head(3)

In [ ]:
df['is_present_tense'] = df['clean_def'].progress_apply(lambda x: detect_tense(x, 'present'))
df.head(3)

### Check proper case

In [ ]:
df[['not_starting_upper_case', 'count_not_starting_uppercase', 'uppercase_in_between', 'count_uppercase_between']] = df['CDE Definition'].progress_apply(lambda x: check_propercase(x))
df.head(3)

### Grammatical check

In [ ]:
tool = LanguageTool('en-US')

#df[['Mistakes','Corrections']] = df['clean_def'].progress_apply(lambda x: language_check(x, tool))
df[['Mistakes','count_mistakes','Corrections']] = df['CDE Definition'].progress_apply(lambda x: language_check(x, tool))

In [ ]:
df.head(3)

In [ ]:
import pickle
with open('grammar_check_archer_result.pkl', 'wb') as wfile:
    pickle.dump(df, wfile)

In [ ]:
import pickle
file = open("grammar_check_archer_result.pkl",'rb')
df = pickle.load(file)
df.head(3)

### Similarity check

**Please refer [new method](#new_similarity)**

In [ ]:
df['score'] = None
df['test_term'] = None
df['test_def'] = None
df['test_id'] = None

In [ ]:
df2 = df.copy()
df2.shape

In [ ]:
cols = list(df.columns)
cols.extend(['test_id','score','test_term','test_def'])
result = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df))):

    test_id = df.loc[i, 'CDE No']
    test_term = df.loc[i, 'CDE Name']
    test_def = df.loc[i, 'CDE Definition']
    test_lemmatized = df.loc[i, 'lemmatized']

    df2['score'] = df['lemmatized'].apply(lambda x: jaccard_similarity(x, test_lemmatized))
    df2['test_term'] = test_term
    df2['test_def'] = test_def
    df2['test_id'] = test_id

    df_ = df2.sort_values('score', ascending=False)
    df_.reset_index(drop=True, inplace=True)

    # store values
    df.loc[i, 'score'] = df_.loc[1, 'score']
    df.loc[i, 'test_term'] = df_.loc[1, 'CDE Name']
    df.loc[i, 'test_def'] = df_.loc[1, 'CDE Definition']
    df.loc[i, 'test_id'] = df_.loc[1, 'CDE No']

    # if i in (114, 234):
    #     store_df = df.copy()
    #     store_df2 = df2.copy()

    # if i==0:
    #     result = df.iloc[[1],:]
    # else:
    #     result = pd.concat([result, df.iloc[[1],:]], axis=0, ignore_index=True)

df[['CDE No','CDE Name','CDE Definition','score','test_term','test_def','test_id']].head(3)
    

#### New method

In [ ]:
# load data
import pickle
file = open("grammar_check_archer_result.pkl",'rb')
df = pickle.load(file)
df.shape

In [ ]:
# df['score'] = None
# df['test_term'] = None
# df['test_def'] = None
# df['test_id'] = None

In [ ]:
# result_cols = ['CDE No','CDE Name','CDE Definition']
result_cols = df.columns.to_list()
# result_df = pd.DataFrame(columns=df.columns.to_list())
result_df = pd.DataFrame(columns=result_cols)

In [ ]:
for i in range(len(df)):

    test_id = df.loc[i, 'CDE No']
    # test_term = df.loc[i, 'CDE Name']
    # test_def = df.loc[i, 'CDE Definition']
    test_lemmatized = df.loc[i, 'lemmatized']

    dup_df = df.copy()
    dup_df['score'] = df['lemmatized'].apply(lambda x: jaccard_similarity(x, test_lemmatized))

    # drop same record
    dup_df.drop(dup_df[dup_df['CDE No']==test_id].index, inplace=True)

    # descending order
    dup_df = dup_df.sort_values('score', ascending=False)
    dup_df.reset_index(drop=True, inplace=True)

    # assign values
    # for col in df.columns:
    for col in result_cols:
        result_df.loc[i, col] = df.loc[i, col]
    
    result_df.reset_index(drop=True, inplace=True)
    result_df.loc[i, 'score'] = dup_df.loc[0, 'score']
    result_df.loc[i, 'test_id'] = dup_df.loc[0, 'CDE No']
    result_df.loc[i, 'test_term'] = dup_df.loc[0, 'CDE Name']
    result_df.loc[i, 'test_def'] = dup_df.loc[0, 'CDE Definition']
    result_df.reset_index(drop=True, inplace=True)

result_df.head(3)

In [ ]:
import pickle
with open('final_archer_top1_result.pkl', 'wb') as wfile:
    pickle.dump(result_df, wfile)

### Top N similarities

In [ ]:
import pickle
file = open("grammar_check_archer_result.pkl",'rb')
df = pickle.load(file)
df.shape

In [ ]:
df.head()

In [ ]:
TOP_N = 1

In [ ]:
df_ = df.copy()

df_['score'] = None
df_['test_term'] = None
df_['test_def'] = None
df_['test_id'] = None

In [ ]:
df2 = df_.copy()
df2.shape

In [ ]:
cols = list(df_.columns)
cols.extend(['test_id','score','test_term','test_def'])

In [ ]:
result = pd.DataFrame(columns=cols)
#temp = result.copy()

for i in tqdm(range(len(df_))):

    temp = pd.DataFrame(columns=cols)
    
    test_id = df_.loc[i, 'CDE No']
    test_term = df_.loc[i, 'CDE Name']
    test_def = df_.loc[i, 'CDE Definition']
    test_lemmatized = df_.loc[i, 'lemmatized']

    df2['score'] = df_['lemmatized'].apply(lambda x: jaccard_similarity(x, test_lemmatized))
    df2['test_term'] = test_term
    df2['test_def'] = test_def
    df2['test_id'] = test_id

    df2 = df2.sort_values('score', ascending=False)
    df2.reset_index(drop=True, inplace=True)

    # store values
    # df.loc[i, 'score'] = df2.loc[1, 'score']
    # df.loc[i, 'test_term'] = df2.loc[1, 'test_term']
    # df.loc[i, 'test_def'] = df2.loc[1, 'test_def']

    # store values
    for n in range(TOP_N):
        for col in df2.columns:
            temp.loc[n, col] = df2.loc[n+1, col]

    result = pd.concat([result, temp], axis=0, ignore_index=True)

    # if i==0:
    #     result = df.iloc[[1],:]
    # else:
    #     result = pd.concat([result, df.iloc[[1],:]], axis=0, ignore_index=True)

result.head(3)
    

In [ ]:
result.shape

In [ ]:
result[['CDE No','CDE Name','CDE Definition','score','test_term','test_def','test_id']]

In [ ]:
import pickle
with open('final_archer_result_topN.pkl', 'wb') as wfile:
    pickle.dump(result, wfile)

#### <a id='new_similarity'>New method</a>

In [ ]:
import pickle
file = open("grammar_check_archer_result.pkl",'rb')
df = pickle.load(file)
df.shape

In [ ]:
df.head()

In [ ]:
TOP_N = 3
# TOP_N = 1

In [ ]:
# result_cols = ['CDE No','CDE Name','CDE Definition']
result_cols = df.columns.to_list()
# result_df = pd.DataFrame(columns=df.columns.to_list())
result_df = pd.DataFrame(columns=result_cols)

In [ ]:
result_row=0
for i in range(len(df)):

    test_id = df.loc[i, 'CDE No']
    test_term = df.loc[i, 'CDE Name']
    test_def = df.loc[i, 'CDE Definition']
    test_lemmatized = df.loc[i, 'lemmatized']

    dup_df = df.copy()
    dup_df['score'] = df['lemmatized'].apply(lambda x: jaccard_similarity(x, test_lemmatized))

    # drop same record
    dup_df.drop(dup_df[dup_df['CDE No']==test_id].index, inplace=True)

    # descending order
    dup_df = dup_df.sort_values('score', ascending=False)
    dup_df.reset_index(drop=True, inplace=True)

    # assign values
    # result_row = len(result_df)+1
    # for col in result_cols:
    #     for row in range(TOP_N):
    #         result_df.loc[result_row, col] = df.loc[i, col]
    #     result_row+=1

    temp_df = pd.DataFrame(columns=result_cols)
    append_list = [temp_df]
    for _ in range(TOP_N):
        append_list.append(df.loc[i,result_cols].to_frame().T)
    temp_df = pd.concat(append_list, ignore_index=True)
    # temp_df.reset_index(drop=True, inplace=True)
    
    for row in range(TOP_N):
        temp_df.loc[row, 'score'] = dup_df.loc[row, 'score']
        temp_df.loc[row, 'test_id'] = dup_df.loc[row, 'CDE No']
        temp_df.loc[row, 'test_term'] = dup_df.loc[row, 'CDE Name']
        temp_df.loc[row, 'test_def'] = dup_df.loc[row, 'CDE Definition']

    result_df = pd.concat([result_df, temp_df], axis=0)
    result_df.reset_index(drop=True, inplace=True)


    # result_df.reset_index(drop=True, inplace=True)
    # for row in range(TOP_N):
    #     result_df.loc[result_row - (TOP_N+1) + row, 'score'] = dup_df.loc[row, 'score']
    #     result_df.loc[result_row - (TOP_N+1) + row, 'test_id'] = dup_df.loc[row, 'CDE No']
    #     result_df.loc[result_row - (TOP_N+1) + row, 'test_term'] = dup_df.loc[row, 'CDE Name']
    #     result_df.loc[result_row - (TOP_N+1) + row, 'test_def'] = dup_df.loc[row, 'CDE Definition']
    # result_df.reset_index(drop=True, inplace=True)

result_df.head()

In [ ]:
result_df.shape

## Save to Excel file

In [ ]:
VERSION = current_time()
# df.to_excel(OUTPUT_PATH+'\\'+f'final_result_{VERSION}.xlsx', index=False)
# df.to_excel(OUTPUT_PATH+'\\'+f'final_archer_result_{VERSION}.xlsx', index=False)
result_df.to_excel(OUTPUT_PATH+'\\'+f'final_archer_top1_result_{VERSION}.xlsx', index=False)

In [ ]:
VERSION = current_time()
result_df.to_excel(OUTPUT_PATH+'\\'+f'final_archer_result_topN_{VERSION}.xlsx', index=False)

## Analysis

## Count words

In [ ]:
word_count = {}
for lem in tqdm(df["lemmatized"].values):
    
    for token in lem:
        if token not in word_count:
            word_count[token] = 1
        else:
            word_count[token] += 1

In [ ]:
word_count

### Plot word count

In [ ]:
word_count = dict(sorted(word_count.items(), key=lambda x: x[1], reverse=True))


In [ ]:
labels = list(word_count.keys())[:20]
plt.figure(figsize=(10,6))
ax1 = plt.subplot()
ax1.set_xticklabels(labels, rotation=45)
plt.bar(list(word_count.keys())[:20], list(word_count.values())[:20])
plt.title("Top 20 terms")
plt.xlabel("Terms")
plt.ylabel("Counts")
plt.show()

## N-Gram

In [ ]:
def n_grams(tokens, n):
    # tokens = vals.split()
    n_gram = [tokens[i:i+n] for i in range(len(tokens)-n+1)]
    return [' '.join(token) for token in n_gram]

In [ ]:
two_gram= df['lemmatized'].progress_apply(lambda x: n_grams(x, 2))

In [ ]:
word_count = {}
for gram in tqdm(two_gram.values):
    
    for token in gram:
        if token not in word_count:
            word_count[token] = 1
        else:
            word_count[token] += 1

word_count = dict(sorted(word_count.items(), key=lambda x: x[1], reverse=True))


In [ ]:
labels = list(word_count.keys())[:10]
plt.figure(figsize=(10,6))
ax1 = plt.subplot()
ax1.set_xticklabels(labels, rotation=45)
plt.title("Top 10 terms")
plt.xlabel("Terms")
plt.ylabel("Counts")
plt.bar(list(word_count.keys())[:10], list(word_count.values())[:10])